In [1]:
# TF-IDF + Logistic Regression

import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_auc_score

In [2]:
# Load data
train = pd.read_csv("/kaggle/input/voai-2025-hsgs-nlp/train.csv")
test = pd.read_csv("/kaggle/input/voai-2025-hsgs-nlp/test.csv")

In [3]:
train.head(2)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0


In [4]:
test.head(2)

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...


In [5]:
# Clean text
def clean_text(text):
    text = str(text).lower()
    return text

train["comment_text"] = train["comment_text"].apply(clean_text)
test["comment_text"] = test["comment_text"].apply(clean_text)

In [6]:
# TF-IDF Vectorization
vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(train["comment_text"])
X_test = vectorizer.transform(test["comment_text"])

In [ ]:
# Logistic Regression
target_cols = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y_train = train[target_cols]

model = OneVsRestClassifier(LogisticRegression(max_iter=2000, C=3, solver="liblinear"))
model.fit(X_train, y_train)

OneVsRestClassifier(estimator=LogisticRegression(C=4, max_iter=5000,
                                                 solver='liblinear'))

In [8]:
preds = model.predict_proba(X_test)

submission = pd.DataFrame(preds, columns=target_cols)
submission.insert(0, "id", test["id"])
submission.to_csv("submission.csv", index=False)